In [8]:
%%writefile workers.py
import torch
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import ThreadedPdfPipelineOptions, RapidOcrOptions, PdfBackend
from docling.datamodel.accelerator_options import AcceleratorDevice, AcceleratorOptions

def gpu_worker(paper_chunk, gpu_id):
    device_str = f"cuda:{gpu_id}"
    
    acc_options = AcceleratorOptions(device=device_str, num_threads=4)
    pipe_options = ThreadedPdfPipelineOptions(
        accelerator_options=acc_options,
        ocr_batch_size=32,      
        pdf_backend=PdfBackend.PYPDFIUM2,
        do_ocr=True,
        ocr_options=RapidOcrOptions(backend="torch")
    )
    
    converter = DocumentConverter(format_options={"pdf": PdfFormatOption(pipeline_options=pipe_options)})
    
    # NEW: We store results as a list of objects instead of one big string
    processed_papers = []
    
    for paper in paper_chunk:
        try:
            print(f"GPU {gpu_id} reading: {paper['title'][:50]}...")
            result = converter.convert(paper['link'])
            content = result.document.export_to_markdown()
            
            # Package the content with its citation metadata
            processed_papers.append({
                "title": paper['title'],
                "link": paper['link'],
                "source": paper.get('source', 'Unknown'),
                "text": content
            })
            torch.cuda.empty_cache() 
        except Exception as e:
            print(f"GPU {gpu_id} error on {paper['title'][:20]}: {e}")
            continue
            
    return processed_papers

Writing workers.py


In [12]:
import warnings
# Tells Python to stay quiet about "FutureWarnings"
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import re
import time
import arxiv
import requests
import multiprocessing as mp
import vertexai
from vertexai.generative_models import GenerativeModel
from workers import gpu_worker 

# --- CONFIGS ---
PROJECT_ID = "dulcet-asset-208017" 
SS_API_KEY = "VA0cJSDyrN9ExLvIPBns74HhMxuCPYuy6v11bJzS"
LOCATION = "us-central1"
TOP_K_READ = 10 

# Initialize Google Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# --- SEARCH ENGINE ---
def unified_scourer(query, ss_key, max_results=40):
    all_results = []
    seen_titles = set()
    print(f"🔍 Scouring ArXiv & Semantic Scholar for: {query}...")
    
    # ArXiv
    client = arxiv.Client()
    for r in client.results(arxiv.Search(query=query, max_results=max_results)):
        t_norm = "".join(filter(str.isalnum, r.title.lower()))
        all_results.append({"title": r.title, "link": r.pdf_url, "abstract": r.summary})
        seen_titles.add(t_norm)

    # Semantic Scholar
    ss_url = f"https://api.semanticscholar.org/graph/v1/paper/search?query={query}&limit={max_results}&fields=title,openAccessPdf,abstract"
    try:
        res = requests.get(ss_url, headers={"x-api-key": ss_key}).json()
        for p in res.get('data', []):
            t_norm = "".join(filter(str.isalnum, p['title'].lower()))
            pdf = (p.get('openAccessPdf') or {}).get('url')
            if t_norm not in seen_titles and pdf:
                all_results.append({"title": p['title'], "link": pdf, "abstract": p.get('abstract', '')})
    except: pass
    return all_results

# --- VERTEX AI JUDGE ---
def judge_papers(candidates, topic):
    """Uses Gemini 2.5 Flash-Lite (the 2026 standard) for fast filtering."""
    print(f"⚖️ Judging {len(candidates)} candidates via Vertex AI (Gemini 2.5)...")
    
    judge_model = GenerativeModel("gemini-2.5-flash-lite")
    winners = []
    
    for paper in candidates[:20]:
        prompt = (f"Topic: {topic}\nPaper: {paper['title']}\n"
                  f"Abstract: {paper['abstract'][:500]}\n"
                  "Is this paper highly relevant? Reply ONLY 'YES' or 'NO'.")
        try:
            resp = judge_model.generate_content(prompt)
            if "YES" in resp.text.upper():
                winners.append(paper)
        except Exception as e:
            print(f"⚠️ Judging error: {e}")
            continue
        if len(winners) >= TOP_K_READ: break
    return winners
    
# --- THE MISSION CONTROL ---
def run_research_mission(topic_name):
    start_total = time.time()
    
    # 1. Triage
    candidates = unified_scourer(topic_name, SS_API_KEY)
    winners = judge_papers(candidates, topic_name)
    
    if not winners:
        print(f"No relevant papers found.")
        return None

    # 2. Parallel Shredder (Local T4 GPUs)
    mid = len(winners) // 2
    chunks = [winners[:mid], winners[mid:]]
    print(f"🚀 Deploying 2x T4 GPUs to shred {len(winners)} papers...")
    
    ctx = mp.get_context('spawn')
    with ctx.Pool(processes=2) as pool:
        worker_outputs = pool.starmap(gpu_worker, [(chunks[0], 0), (chunks[1], 1)])
    
    all_docs = [doc for sublist in worker_outputs for doc in sublist]
    
    # 3. Citation & Corpus Map
    full_corpus = ""
    bibliography = "\n\n# References\n"
    for i, doc in enumerate(all_docs):
        ref_num = i + 1
        full_corpus += f"\n--- DATA FROM SOURCE [{ref_num}] ---\n{doc['text']}\n"
        bibliography += f"[{ref_num}] {doc['title']}. Link: {doc['link']}\n"
    
    # 4. Vertex AI Synthesis (Llama 3.3 70B with Literature Matrix)
    print(f"Synthesizing Narrative & Literature Matrix via Llama 3.3 70B...")

    LLAMA_PATH = f"projects/{PROJECT_ID}/locations/{LOCATION}/publishers/meta/models/llama-3.3-70b-instruct-maas"
    writer_model = GenerativeModel(LLAMA_PATH)
    
    system_instr = (
        f"You are a Senior Silicon Architect. Analyze the data into a 2000-word deep-dive on {topic_name}.\n"
        "Use in-text citations like [1] or [2] for every technical fact based ONLY on 'DATA FROM SOURCE [X]' headers.\n\n"
        "### REQUIRED ADD-ON: LITERATURE SYNTHESIS MATRIX\n"
        "At the very end of your report, provide a structured Markdown table summarizing the current work found in the sources. "
        "Use these columns:\n"
        "| Source [Ref] | Research Objective | Hardware/Methodology | Work Done / Key Findings |\n"
        "| :--- | :--- | :--- | :--- |\n"
        "Ensure the 'Work Done' column specifically addresses the query topic."
    )
    
    final_prompt = f"{system_instr}\n\nHardware Raw Data (Source Material):\n{full_corpus[:38000]}"
    
    try:
        report = writer_model.generate_content(final_prompt).text
    except Exception as e:
        print(f"Synthesis Error: {e}")
        return None
    
    # 5. Save & Finish
    final_output = report + bibliography
    filename = f"{topic_name.replace(' ', '_')}_Synthesis_Matrix.md"
    with open(filename, "w") as f:
        f.write(final_output)
    
    print(f"\n✅ MISSION COMPLETE. Time: {round((time.time()-start_total)/60, 2)}m. Saved to {filename}")
    return final_output

In [13]:
# Try a high-end hardware comparison
run_research_mission("NVIDIA Hopper")

2026-01-09 18:42:44,704 - INFO - Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=NVIDIA+Hopper&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100


🔍 Scouring ArXiv & Semantic Scholar for: NVIDIA Hopper...


2026-01-09 18:42:45,499 - INFO - Got first page: 100 of 3117 total results


⚖️ Judging 43 candidates via Vertex AI (Gemini 2.5)...


/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


🚀 Deploying 2x T4 GPUs to shred 7 papers...


2026-01-09 18:42:59,085 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 18:42:59,099 - INFO - Going to convert document batch...
2026-01-09 18:42:59,099 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e4c42daca97a8e21534d8c76e00da367
2026-01-09 18:42:59,110 - INFO - Loading plugin 'docling_defaults'
2026-01-09 18:42:59,112 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-09 18:42:59,121 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 18:42:59,123 - INFO - Loading plugin 'docling_defaults'
2026-01-09 18:42:59,125 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-09 18:42:59,139 - INFO - Going to convert document batch...
2026-01-09 18:42:59,139 - INFO - Initializing pipeline for StandardPdfPipeline with options hash bcaac49b74d25edc113eed3b5c5ed5f6
2026-01-09 18:42:59,151 - INFO - Loading plugin 'docling_defaults'
2026-01-09 18:42:59,152 - INFO - Register

GPU 1 reading: A Case Study in CUDA Kernel Fusion: Implementing F...
GPU 1 reading: Automatic BLAS Offloading on Unified Memory Archit...
GPU 1 reading: Hopper flows of deformable particles...
GPU 1 reading: NVIDIA GPU Confidential Computing Demystified...
Synthesizing Narrative & Literature Matrix via Llama 3.3 70B...

✅ MISSION COMPLETE. Time: 1.01m. Saved to NVIDIA_Hopper_Synthesis_Matrix.md


"As a Senior Silicon Architect, I will provide a deep-dive analysis of NVIDIA Hopper, a cutting-edge GPU architecture designed to accelerate artificial intelligence (AI) and high-performance computing (HPC) workloads.\n\n**Introduction**\n\nNVIDIA Hopper is the latest GPU architecture from NVIDIA, designed to provide significant performance and power efficiency improvements over its predecessors, Ampere and Ada Lovelace [1]. Hopper introduces several innovative features, including new tensor cores, dynamic programming X (DPX) instructions, distributed shared memory, and an enhanced asynchronous execution mechanism.\n\n**Tensor Cores**\n\nTensor cores are a critical component of modern GPU architectures, responsible for accelerating deep learning and AI workloads [2]. Hopper introduces new tensor cores that support FP8, DPX, and distributed shared memory. The tensor cores are designed to provide higher performance and efficiency for AI workloads, with support for mixed-precision computi

In [3]:
run_research_mission("NVIDIA Blackwell NVLink Speeds")

2026-01-09 15:59:28,902 - INFO - Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=NVIDIA+Blackwell+NVLink+Speeds&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100


Scouring ArXiv & Semantic Scholar...


2026-01-09 15:59:30,035 - INFO - Got first page: 100 of 71759 total results


Judging 43 candidates for architectural density...
Deploying 2x T4 GPUs for 20 papers...


2026-01-09 15:59:39,982 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 15:59:39,996 - INFO - Going to convert document batch...
2026-01-09 15:59:39,997 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e4c42daca97a8e21534d8c76e00da367
2026-01-09 15:59:40,008 - INFO - Loading plugin 'docling_defaults'
2026-01-09 15:59:40,009 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-09 15:59:40,020 - INFO - Loading plugin 'docling_defaults'
2026-01-09 15:59:40,022 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-09 15:59:40,907 - INFO - Accelerator device: 'cuda:0'
[INFO] 2026-01-09 15:59:40,938 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-09 15:59:40,940 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-09 15:59:40,980 [RapidOCR] download_file.py:60: File exists and is valid: /opt/conda/lib/python3.10/site-packages/rapidocr/models/ch_PP-O

🔥 GPU 1 active on: An Open, Programmable, Multi-vendor 5G O...
🔥 GPU 1 active on: vDNN: Virtualized Deep Neural Networks f...
🔥 GPU 1 active on: Beyond discounted returns: Robust Markov...
🔥 GPU 1 active on: Application of Graphics Processing Units...
🔥 GPU 1 active on: Audio2Face-3D: Audio-driven Realistic Fa...
🔥 GPU 1 active on: Rao-Blackwellizing Field Goal Percentage...
🔥 GPU 1 active on: Causality and the speed of sound...
🔥 GPU 1 active on: Cosmos-Transfer1: Conditional World Gene...
🔥 GPU 1 active on: Reducing Blackwell and Average Optimalit...
🔥 GPU 1 active on: Evaluating Modern GPU Interconnect: PCIe...
Synthesizing Meta-Report on NVIDIA Blackwell NVLink Speeds via Llama 3.3 70B...

MISSION COMPLETE. Total Time: 2.5 minutes.
Report saved to NVIDIA_Blackwell_NVLink_Speeds_Report.md


"I'd be happy to provide a deep-dive analysis of NVIDIA Blackwell NVLink speeds. However, I must point out that the provided text appears to be unrelated to the topic of NVIDIA Blackwell NVLink speeds. The text seems to be discussing Homomorphic Secret Sharing (HSS) schemes, which is a cryptography-related topic.\n\nAssuming that you'd like me to provide a general analysis of NVIDIA Blackwell NVLink speeds, I'll provide an overview of the topic.\n\n**Introduction**\n\nNVIDIA's NVLink is a high-speed interconnect technology designed to provide low-latency and high-bandwidth communication between NVIDIA graphics processing units (GPUs) and other components such as central processing units (CPUs) and memory. The Blackwell architecture is a specific implementation of NVLink, which is used in NVIDIA's datacenter-focused products, such as the NVIDIA A100 GPU.\n\n**Interconnects**\n\nNVLink is a serial, point-to-point interconnect that uses a differential signaling scheme to transmit data ove

In [4]:
run_research_mission("NVIDIA H100")

2026-01-09 16:03:26,135 - INFO - Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=NVIDIA+H100&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100


Scouring ArXiv & Semantic Scholar...


2026-01-09 16:03:27,303 - INFO - Got first page: 100 of 2929 total results


Judging 49 candidates for architectural density...
Deploying 2x T4 GPUs for 20 papers...


2026-01-09 16:03:37,503 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 16:03:37,545 - INFO - Going to convert document batch...
2026-01-09 16:03:37,546 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e4c42daca97a8e21534d8c76e00da367
2026-01-09 16:03:37,549 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 16:03:37,558 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:03:37,559 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-09 16:03:37,572 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:03:37,572 - INFO - Going to convert document batch...
2026-01-09 16:03:37,572 - INFO - Initializing pipeline for StandardPdfPipeline with options hash bcaac49b74d25edc113eed3b5c5ed5f6
2026-01-09 16:03:37,574 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-09 16:03:37,584 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:03:37,585 - INFO - Register

Synthesizing Meta-Report on NVIDIA H100 via Llama 3.3 70B...

MISSION COMPLETE. Total Time: 3.4 minutes.
Report saved to NVIDIA_H100_Report.md


"**Introduction to NVIDIA H100**\n\nThe NVIDIA H100 is a cutting-edge GPU accelerator designed to deliver unprecedented AI compute throughput, optimized transformer acceleration, and energy efficiency improvements. Built on the Hopper architecture, the H100 GPU provides a significant boost in performance and power efficiency compared to its predecessors. In this analysis, we will delve into the interconnects, chiplet layout, and memory throughput of the NVIDIA H100, with a focus on its applications in deep learning and artificial intelligence.\n\n**Interconnects: NVLink and PCI Express**\n\nThe NVIDIA H100 features a high-speed interconnect called NVLink, which provides a bandwidth of up to 900 GB/s. NVLink is a proprietary interconnect technology developed by NVIDIA, designed to enable high-speed communication between GPUs, CPUs, and other components. The H100 also supports PCI Express (PCIe) 4.0, which offers a maximum bandwidth of 32 GB/s per lane. The combination of NVLink and PCIe

In [6]:
if __name__ == "__main__":
    run_research_mission("NVIDIA 5090")

2026-01-09 16:07:48,691 - INFO - Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=NVIDIA+5090&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100


Scouring ArXiv & Semantic Scholar...


2026-01-09 16:07:50,064 - INFO - Got first page: 100 of 2874 total results


Judging 56 candidates for architectural density...
Deploying 2x T4 GPUs for 20 papers...


2026-01-09 16:08:00,464 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 16:08:00,509 - INFO - Going to convert document batch...
2026-01-09 16:08:00,509 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e4c42daca97a8e21534d8c76e00da367
2026-01-09 16:08:00,510 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 16:08:00,521 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:08:00,522 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-09 16:08:00,530 - INFO - Going to convert document batch...
2026-01-09 16:08:00,530 - INFO - Initializing pipeline for StandardPdfPipeline with options hash bcaac49b74d25edc113eed3b5c5ed5f6
2026-01-09 16:08:00,533 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:08:00,536 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-09 16:08:00,541 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:08:00,542 - INFO - Register

🔥 GPU 0 active on: Cosmos-Transfer1: Conditional World Gene...
🔥 GPU 0 active on: NVIDIA Tensor Core Programmability, Perf...
🔥 GPU 0 active on: JExplore: Design Space Exploration Tool ...
🔥 GPU 0 active on: A GEMM interface and implementation on N...
🔥 GPU 0 active on: A Modular AI-Driven Intrusion Detection ...
⚠️ GPU 0 error on A Modular AI-Driven : 403 Client Error: Forbidden for url: https://www.mdpi.com/1424-8220/25/1/130
🔥 GPU 0 active on: NVIDIA at the Center of the Generative A...
⚠️ GPU 0 error on NVIDIA at the Center: 403 Client Error: Forbidden for url: https://dl.acm.org/doi/pdf/10.1145/3631537
🔥 GPU 0 active on: NVIDIA Nemotron Nano 2: An Accurate and ...
🔥 GPU 0 active on: Alpamayo-R1: Bridging Reasoning and Acti...
🔥 GPU 0 active on: Part-time Power Measurements: nvidia-smi...
🔥 GPU 0 active on: Hierarchical Roofline Analysis: How to C...
Synthesizing Meta-Report on NVIDIA 5090 via Llama 3.3 70B...

MISSION COMPLETE. Total Time: 2.26 minutes.
Report saved to NVIDIA_5090

In [10]:
if __name__ == "__main__":
    run_research_mission("Intel Arc B-Series")

2026-01-09 16:14:32,829 - INFO - Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=Intel+Arc+B-Series&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100


Scouring ArXiv & Semantic Scholar...


2026-01-09 16:14:33,925 - INFO - Got first page: 100 of 15855 total results


Judging 52 candidates for architectural density...
Deploying 2x T4 GPUs for 20 papers...


2026-01-09 16:14:44,287 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 16:14:44,300 - INFO - Going to convert document batch...
2026-01-09 16:14:44,300 - INFO - Initializing pipeline for StandardPdfPipeline with options hash bcaac49b74d25edc113eed3b5c5ed5f6
2026-01-09 16:14:44,311 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:14:44,313 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-09 16:14:44,325 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:14:44,327 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-09 16:14:44,340 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 16:14:44,352 - INFO - Going to convert document batch...
2026-01-09 16:14:44,352 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e4c42daca97a8e21534d8c76e00da367
2026-01-09 16:14:44,364 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:14:44,366 - INFO - Register

GPU 1 reading: Sharpening and generalizations of Carlson's double...
GPU 1 reading: The Sums of a Double Hypergeometric Series and of ...
GPU 1 reading: Leveraging Intel SGX to Create a Nondisclosure Cry...
GPU 1 reading: Subclasses of Normal Helly Circular-Arc Graphs...
GPU 1 reading: Concise sharpening and generalizations of Shafer's...
GPU 1 reading: Intel MPX Explained: An Empirical Study of Intel M...
GPU 1 reading: Distributed Transfer Learning with 4th Gen Intel X...
GPU 1 reading: Elzar: Triple Modular Redundancy using Intel Advan...
GPU 1 reading: The Lensed Arc Production Efficiency of Galaxy Clu...
GPU 1 reading: Algebraic structure of aromatic B-series...
Synthesizing Cited Meta-Report on Intel Arc B-Series...


HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/groq/openai/v1/chat/completions (Request ID: Root=1-696129bf-2745d3df4d6dce1d3339f088;3122752f-13e4-4a8c-b9db-80217242db74)

You have reached the free monthly usage limit for groq. Subscribe to PRO to get 20x more included usage, or add pre-paid credits to your account.

In [ ]:
import accelerate
import transformers
from docling.document_converter import DocumentConverter

print(f"Accelerate version: {accelerate.__version__}")
print(f"Transformers version: {transformers.__version__}")
print("✅ Imports successful!")

In [ ]:
!pip install docling arxiv requests huggingface_hub

In [ ]:
!pip install accelerate>=0.26.0 transformers>=4.38.0

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [ ]:
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124

In [ ]:
!pip install docling arxiv requests huggingface_hub

In [ ]:
!nvidia-smi

In [11]:
!pip install --upgrade google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.1 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
      Successfully uninstalled google-auth-2.43.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [google-auth]  WARNING: Failed to remove contents in a temporary directory '/opt/conda/lib/python3.10/site-packages/google/~uth'.
  You can safely remove it manually.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [google-auth]  WARNING: Failed to remove contents in a temporary directory '/opt/conda/lib/python3.10/site-packages/google/~auth2'.
  You can safely remove it manually.
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.130.0oogle-auth]
    Uninstalling google-cloud-aiplatform-1.130.0:━━━━━━━━━━━━━━━━━ 1/2 [google-cloud-aiplatform]
      Successfully uninstalled google-cloud-aiplatform-1.130.0━━━━ 1/2 [google-cloud-a

In [1]:
# Force-upgrade the auth and aiplatform libraries to be in sync
!pip install --upgrade google-auth google-cloud-aiplatform --user